# .NET 9 Clean Architecture 三層式專案 + Vue3 Vite 前後端分離 CRUD 實作

## 1. 建立 .NET 9 專案與資料夾結構

使用 dotnet CLI 建立 ASP.NET Core Web API 專案，並依 Clean Architecture 與三層式架構規劃資料夾：
- `DrinkShop.Api`：API 層，Controller、Middleware、Response、DI 註冊
- `DrinkShop.Application`：應用層，Service、DTO、介面
- `DrinkShop.Domain`：領域層，Entity、介面
- `DrinkShop.Infrastructure`：基礎設施層，Repository、DbContext

```bash
# 建立解決方案與專案
# (已於本專案完成)
dotnet new sln -n DrinkShop

dotnet new webapi -n DrinkShop.Api --framework net9.0 --use-minimal-apis

dotnet new classlib -n DrinkShop.Application --framework net9.0

dotnet new classlib -n DrinkShop.Domain --framework net9.0

dotnet new classlib -n DrinkShop.Infrastructure --framework net9.0

# 加入專案參考
# (Api 參考 Application/Domain/Infrastructure，Application/Infrastructure 參考 Domain)
dotnet sln add DrinkShop.Api/DrinkShop.Api.csproj
# ... 依序加入其他專案 ...
```

資料夾結構範例：
```
DrinkShop.Api/
  Controllers/
  Middlewares/
  Responses/
DrinkShop.Application/
  Interfaces/
  Services/
DrinkShop.Domain/
  Entities/
DrinkShop.Infrastructure/
  Repositories/
```


## 2. 定義資料模型與 DbContext（使用 SQLite）

1. 在 `Domain/Entities` 建立資料實體（如 Drink）。
2. 在 `Infrastructure` 建立 DbContext，並於 `appsettings.json` 設定 SQLite 連線字串。
3. 於 `Program.cs` 註冊 DbContext，並依環境載入不同設定。

**Drink 實體範例**
```csharp
// DrinkShop.Domain/Entities/Drink.cs
public class Drink
{
    public int Id { get; set; }
    public string Name { get; set; } = string.Empty;
    public decimal Price { get; set; }
}
```

**DbContext 範例**
```csharp
// DrinkShop.Infrastructure/DrinkShopDbContext.cs
using Microsoft.EntityFrameworkCore;
using DrinkShop.Domain.Entities;

public class DrinkShopDbContext : DbContext
{
    public DrinkShopDbContext(DbContextOptions<DrinkShopDbContext> options) : base(options) { }
    public DbSet<Drink> Drinks => Set<Drink>();
}
```

**appsettings.json 範例**
```json
{
  "ConnectionStrings": {
    "DefaultConnection": "Data Source=drinkshop.db"
  }
}
```

**Program.cs 註冊 DbContext**
```csharp
builder.Services.AddDbContext<DrinkShopDbContext>(options =>
    options.UseSqlite(builder.Configuration.GetConnectionString("DefaultConnection")));
```


## 3. 實作 Repository 層與介面隔離

1. 在 `Application/Interfaces` 定義 IRepository 介面與 CRUD 方法。
2. 在 `Infrastructure/Repositories` 實作 Repository，並注入 DbContext。
3. 於 DI 註冊 Repository 實作。

**IRepository 介面範例**
```csharp
// DrinkShop.Application/Interfaces/IDrinkRepository.cs
using DrinkShop.Domain.Entities;

public interface IDrinkRepository
{
    Task<IEnumerable<Drink>> GetAllAsync();
    Task<Drink?> GetByIdAsync(int id);
    Task AddAsync(Drink drink);
    Task UpdateAsync(Drink drink);
    Task DeleteAsync(int id);
}
```

**Repository 實作範例**
```csharp
// DrinkShop.Infrastructure/Repositories/DrinkRepository.cs
using DrinkShop.Application.Interfaces;
using DrinkShop.Domain.Entities;
using Microsoft.EntityFrameworkCore;

public class DrinkRepository : IDrinkRepository
{
    private readonly DrinkShopDbContext _context;
    public DrinkRepository(DrinkShopDbContext context) => _context = context;

    public async Task<IEnumerable<Drink>> GetAllAsync() => await _context.Drinks.ToListAsync();
    public async Task<Drink?> GetByIdAsync(int id) => await _context.Drinks.FindAsync(id);
    public async Task AddAsync(Drink drink) { _context.Drinks.Add(drink); await _context.SaveChangesAsync(); }
    public async Task UpdateAsync(Drink drink) { _context.Drinks.Update(drink); await _context.SaveChangesAsync(); }
    public async Task DeleteAsync(int id)
    {
        var drink = await _context.Drinks.FindAsync(id);
        if (drink != null) { _context.Drinks.Remove(drink); await _context.SaveChangesAsync(); }
    }
}
```

**Program.cs 註冊 Repository**
```csharp
builder.Services.AddScoped<IDrinkRepository, DrinkRepository>();
```


## 4. 實作 Service 層與 DI 建構子注入

1. 在 `Application/Interfaces` 定義 IService 介面（如 IDrinkService）。
2. 在 `Application/Services` 實作 Service，注入 Repository。
3. 於 DI 註冊 Service 實作。

**IDrinkService 介面範例**
```csharp
// DrinkShop.Application/Interfaces/IDrinkService.cs
using DrinkShop.Domain.Entities;

public interface IDrinkService
{
    Task<IEnumerable<Drink>> GetAllAsync();
    Task<Drink?> GetByIdAsync(int id);
    Task AddAsync(Drink drink);
    Task UpdateAsync(Drink drink);
    Task DeleteAsync(int id);
}
```

**DrinkService 實作範例**
```csharp
// DrinkShop.Application/Services/DrinkService.cs
using DrinkShop.Application.Interfaces;
using DrinkShop.Domain.Entities;

public class DrinkService : IDrinkService
{
    private readonly IDrinkRepository _repo;
    public DrinkService(IDrinkRepository repo) => _repo = repo;

    public Task<IEnumerable<Drink>> GetAllAsync() => _repo.GetAllAsync();
    public Task<Drink?> GetByIdAsync(int id) => _repo.GetByIdAsync(id);
    public Task AddAsync(Drink drink) => _repo.AddAsync(drink);
    public Task UpdateAsync(Drink drink) => _repo.UpdateAsync(drink);
    public Task DeleteAsync(int id) => _repo.DeleteAsync(id);
}
```

**Program.cs 註冊 Service**
```csharp
builder.Services.AddScoped<IDrinkService, DrinkService>();
```


## 5. 實作 Controller 層與 API 版本控制

1. 在 `Api/Controllers` 建立 Controller，注入 Service，實作 CRUD API。
2. 設定 API 版本控制（如 v1 路由）。

**DrinkController 範例**
```csharp
// DrinkShop.Api/Controllers/DrinkController.cs
using Microsoft.AspNetCore.Mvc;
using DrinkShop.Application.Interfaces;
using DrinkShop.Domain.Entities;

[ApiController]
[Route("api/v{version:apiVersion}/[controller]")]
[ApiVersion("1.0")]
public class DrinkController : ControllerBase
{
    private readonly IDrinkService _service;
    public DrinkController(IDrinkService service) => _service = service;

    [HttpGet]
    public async Task<IActionResult> Get() => Ok(await _service.GetAllAsync());

    [HttpGet("{id}")]
    public async Task<IActionResult> Get(int id)
        => await _service.GetByIdAsync(id) is Drink d ? Ok(d) : NotFound();

    [HttpPost]
    public async Task<IActionResult> Post([FromBody] Drink drink)
    {
        await _service.AddAsync(drink);
        return CreatedAtAction(nameof(Get), new { id = drink.Id }, drink);
    }

    [HttpPut("{id}")]
    public async Task<IActionResult> Put(int id, [FromBody] Drink drink)
    {
        if (id != drink.Id) return BadRequest();
        await _service.UpdateAsync(drink);
        return NoContent();
    }

    [HttpDelete("{id}")]
    public async Task<IActionResult> Delete(int id)
    {
        await _service.DeleteAsync(id);
        return NoContent();
    }
}
```

**Program.cs 註冊 API 版本控制**
```csharp
builder.Services.AddApiVersioning(options =>
{
    options.DefaultApiVersion = new ApiVersion(1, 0);
    options.AssumeDefaultVersionWhenUnspecified = true;
    options.ReportApiVersions = true;
});
```


## 6. 設定 Clean Architecture 與命名規範

- 各層命名規範：
  - Controller：`DrinkController`（Api/Controllers）
  - Service：`DrinkService`、介面 `IDrinkService`（Application/Services, Application/Interfaces）
  - Repository：`DrinkRepository`、介面 `IDrinkRepository`（Infrastructure/Repositories, Application/Interfaces）
  - Entity：`Drink`（Domain/Entities）
- 原則：
  - 依賴反轉，Api 只依賴 Application，Application 只依賴 Domain，Infrastructure 只依賴 Domain
  - 介面與實作分離，命名明確
  - DI 注入皆用介面

## 7. 統一例外處理 Middleware 與回應格式

1. 在 `Api/Middlewares` 建立 ExceptionMiddleware，攔截全域例外。
2. 統一 API 回應格式（如 code, message, data）。
3. 在 `Program.cs` 註冊 Middleware。

**統一回應格式範例**
```csharp
// DrinkShop.Api/Responses/ApiResponse.cs
public class ApiResponse<T>
{
    public int Code { get; set; }
    public string Message { get; set; } = string.Empty;
    public T? Data { get; set; }
    public ApiResponse(int code, string message, T? data = default)
    {
        Code = code;
        Message = message;
        Data = data;
    }
}
```

**ExceptionMiddleware 範例**
```csharp
// DrinkShop.Api/Middlewares/ExceptionMiddleware.cs
public class ExceptionMiddleware
{
    private readonly RequestDelegate _next;
    private readonly ILogger<ExceptionMiddleware> _logger;
    public ExceptionMiddleware(RequestDelegate next, ILogger<ExceptionMiddleware> logger)
    {
        _next = next;
        _logger = logger;
    }
    public async Task InvokeAsync(HttpContext context)
    {
        try { await _next(context); }
        catch (Exception ex)
        {
            _logger.LogError(ex, ex.Message);
            context.Response.ContentType = "application/json";
            context.Response.StatusCode = 500;
            var response = new ApiResponse<string>(500, "Internal Server Error", ex.Message);
            await context.Response.WriteAsJsonAsync(response);
        }
    }
}
```

**Program.cs 註冊 Middleware**
```csharp
app.UseMiddleware<ExceptionMiddleware>();
```

**Controller 統一回應格式用法**
```csharp
return Ok(new ApiResponse<IEnumerable<Drink>>(200, "Success", drinks));
```


## 8. 設定檔分環境管理

- 使用 `appsettings.Development.json`、`appsettings.Production.json` 管理不同環境設定。
- 於 `Program.cs` 依環境自動載入設定。

**Program.cs 範例**
```csharp
var builder = WebApplication.CreateBuilder(args);
// 預設自動載入 appsettings.{Environment}.json
```

## 9. 自動化 Mapping（AutoMapper）

1. 安裝 AutoMapper 與 DI 擴充套件。
2. 建立 Mapping Profile，於 Service 層自動轉換 DTO 與 Entity。
3. 於 `Program.cs` 註冊 AutoMapper。

**Mapping Profile 範例**
```csharp
// DrinkShop.Application/Mapping/DrinkProfile.cs
using AutoMapper;
using DrinkShop.Domain.Entities;
using DrinkShop.Application.DTOs;

public class DrinkProfile : Profile
{
    public DrinkProfile()
    {
        CreateMap<Drink, DrinkDto>().ReverseMap();
    }
}
```

**Program.cs 註冊 AutoMapper**
```csharp
builder.Services.AddAutoMapper(typeof(DrinkProfile));
```

## 10. Logging 規劃與實作

- 使用 Serilog 或 Microsoft.Extensions.Logging。
- 於 `Program.cs` 註冊 Serilog，並於 Middleware、Service、Repository 層記錄必要資訊。

**Program.cs 註冊 Serilog 範例**
```csharp
using Serilog;

Log.Logger = new LoggerConfiguration()
    .WriteTo.Console()
    .WriteTo.File("logs/log.txt", rollingInterval: RollingInterval.Day)
    .CreateLogger();

builder.Host.UseSerilog();
```

## 11. DbContext 生命周期管理

- 於 DI 註冊 DbContext 為 Scoped（每個 HTTP 請求獨立 DbContext 實例）。

**Program.cs 範例**
```csharp
builder.Services.AddDbContext<DrinkShopDbContext>(options =>
    options.UseSqlite(builder.Configuration.GetConnectionString("DefaultConnection")),
    ServiceLifetime.Scoped);
```


## 12. 防止常見漏洞（如 SQL Injection、XSS）

- 使用 EF Core 預防 SQL Injection（不拼接 SQL，僅用 LINQ/EF 方法）。
- Controller 層驗證輸入資料，避免 XSS（可用 DataAnnotations、FluentValidation）。
- 設定 CORS，僅允許信任來源。

**CORS 設定範例**
```csharp
builder.Services.AddCors(options =>
{
    options.AddPolicy("AllowFrontend", policy =>
        policy.WithOrigins("http://localhost:5173") // Vue3 Vite 預設 port
              .AllowAnyHeader()
              .AllowAnyMethod());
});
app.UseCors("AllowFrontend");
```

## 13. 預留 GitHub CI/CD 與 Azure 部署流程

- 建立 `.github/workflows/ci.yml`，自動化建置、測試、部署。
- 預留 Azure Web App 部署步驟。

**ci.yml 範本**
```yaml
name: .NET CI
on:
  push:
    branches: [ main ]
jobs:
  build:
    runs-on: ubuntu-latest
    steps:
    - uses: actions/checkout@v4
    - name: Setup .NET
      uses: actions/setup-dotnet@v4
      with:
        dotnet-version: '9.0.x'
    - name: Restore
      run: dotnet restore
    - name: Build
      run: dotnet build --no-restore
    - name: Test
      run: dotnet test --no-build --verbosity normal
    # - name: Azure WebApp Deploy
    #   uses: azure/webapps-deploy@v2
    #   with: ...
```

## 14. 建立前端 Vue3 + Vite 專案並串接 API

1. 使用 npm/yarn 建立 Vue3 + Vite 專案。
2. 設定 axios 串接後端 API。

```bash
npm create vite@latest drinkshop-frontend -- --template vue
cd drinkshop-frontend
npm install axios
```

**axios 設定範例**
```js
// src/api.js
import axios from 'axios';
export default axios.create({ baseURL: 'http://localhost:5000/api/v1' });
```

## 15. 實作最簡單的 CRUD 功能

- 後端 Controller 已提供 CRUD API。
- 前端 Vue3 實作 CRUD 頁面，呼叫 API 並顯示資料。

**Vue3 CRUD 元件簡例**
```vue
<template>
  <div>
    <form @submit.prevent="addDrink">
      <input v-model="newDrink.name" placeholder="名稱" />
      <input v-model.number="newDrink.price" placeholder="價格" type="number" />
      <button type="submit">新增</button>
    </form>
    <ul>
      <li v-for="d in drinks" :key="d.id">
        {{ d.name }} - {{ d.price }}
        <button @click="removeDrink(d.id)">刪除</button>
      </li>
    </ul>
  </div>
</template>
<script setup>
import { ref, onMounted } from 'vue';
import api from './api';
const drinks = ref([]);
const newDrink = ref({ name: '', price: 0 });
const fetchDrinks = async () => { drinks.value = (await api.get('/drink')).data.data; };
onMounted(fetchDrinks);
const addDrink = async () => {
  await api.post('/drink', newDrink.value);
  newDrink.value = { name: '', price: 0 };
  fetchDrinks();
};
const removeDrink = async (id) => {
  await api.delete(`/drink/${id}`);
  fetchDrinks();
};
</script>
```
